In [57]:
# Import necessary libraries
import json
import os
from datetime import timedelta

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from pandas.tseries.offsets import MonthEnd
# Set plotting style
sns.set(style='whitegrid')

In [58]:
# Load the data from the text file
data = pd.read_csv('data/raw/PET/PET.txt', sep='\t', header=None, names=['json_str'])

# Function to parse JSON strings
def parse_json_str(json_str):
    return json.loads(json_str)

# Apply the function to parse the JSON strings
parsed_data = data['json_str'].apply(parse_json_str)

# Create a DataFrame from the parsed JSON data
df = pd.json_normalize(parsed_data)

# Display the first few rows of the DataFrame
df.head()

                       series_id  \
0  PET.EMM_EPMPR_PTE_Y35NY_DPG.W   
1  PET.EMM_EPMPR_PTE_Y44HO_DPG.W   
2  PET.EMM_EPMMR_PTE_R5XCA_DPG.W   
3  PET.EMM_EPMMR_PTE_Y05LA_DPG.W   
4  PET.EMM_EPMMR_PTE_Y05SF_DPG.W   

                                                name               units  f  \
0  New York Harbor Premium Reformulated Retail Ga...  Dollars per Gallon  W   
1  Houston, TX Premium Reformulated Retail Gasoli...  Dollars per Gallon  W   
2  West Coast (PADD 5) Except California Midgrade...  Dollars per Gallon  W   
3  Los Angeles, CA Midgrade Reformulated Retail G...  Dollars per Gallon  W   
4  San Francisco, CA Midgrade Reformulated Retail...  Dollars per Gallon  W   

  unitsshort                                        description copyright  \
0      $/gal  New York Harbor Premium Reformulated Retail Ga...      None   
1      $/gal  Houston, TX Premium Reformulated Retail Gasoli...      None   
2      $/gal  West Coast (PADD 5) Except California Midgrade...      None   


In [59]:
df.columns

Index(['series_id', 'name', 'units', 'f', 'unitsshort', 'description',
       'copyright', 'source', 'iso3166', 'geography', 'start', 'end',
       'last_updated', 'data', 'geography2', 'category_id',
       'parent_category_id', 'notes', 'childseries'],
      dtype='object')

In [60]:
# Explode the 'data' column to separate rows for each date-value pair
df = df.explode('data')
df.head(10)

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,start,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240617, 4.306]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240610, 4.328]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240603, 4.367]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240527, 4.412]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240520, 4.399]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240513, 4.434]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240506, 4.457]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240429, 4.469]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240422, 4.472]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240415, 4.224]",NaN,NaN,NaN,NaN,NaN


In [61]:
# Select distinct series_id, name, units
df_series = df[['series_id', 'name', 'units', 'unitsshort']].drop_duplicates()
df_series.head(10)

,series_id,name,units,unitsshort
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,$/gal
1,PET.EMM_EPMPR_PTE_Y44HO_DPG.W,"Houston, TX Premium Reformulated Retail Gasoli...",Dollars per Gallon,$/gal
2,PET.EMM_EPMMR_PTE_R5XCA_DPG.W,West Coast (PADD 5) Except California Midgrade...,Dollars per Gallon,$/gal
3,PET.EMM_EPMMR_PTE_Y05LA_DPG.W,"Los Angeles, CA Midgrade Reformulated Retail G...",Dollars per Gallon,$/gal
4,PET.EMM_EPMMR_PTE_Y05SF_DPG.W,"San Francisco, CA Midgrade Reformulated Retail...",Dollars per Gallon,$/gal
5,PET.EMM_EPM0R_PTE_R5XCA_DPG.W,West Coast (PADD 5) Except California All Grad...,Dollars per Gallon,$/gal
6,PET.EMM_EPM0R_PTE_Y05LA_DPG.W,"Los Angeles, CA All Grades Reformulated Retail...",Dollars per Gallon,$/gal
7,PET.EMM_EPM0R_PTE_Y05SF_DPG.W,"San Francisco, CA All Grades Reformulated Reta...",Dollars per Gallon,$/gal
8,PET.EMM_EPM0R_PTE_Y35NY_DPG.W,New York Harbor All Grades Reformulated Retail...,Dollars per Gallon,$/gal
9,PET.EMM_EPM0R_PTE_Y44HO_DPG.W,"Houston, TX All Grades Reformulated Retail Gas...",Dollars per Gallon,$/gal


In [62]:
# Filter rows based on columns: 'series_id', 'units'
df_series_nonas = df_series[(df_series['series_id'].notna()) & (df_series['units'].notna())]
df_series_nonas.head(10)

,series_id,name,units,unitsshort
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,$/gal
1,PET.EMM_EPMPR_PTE_Y44HO_DPG.W,"Houston, TX Premium Reformulated Retail Gasoli...",Dollars per Gallon,$/gal
2,PET.EMM_EPMMR_PTE_R5XCA_DPG.W,West Coast (PADD 5) Except California Midgrade...,Dollars per Gallon,$/gal
3,PET.EMM_EPMMR_PTE_Y05LA_DPG.W,"Los Angeles, CA Midgrade Reformulated Retail G...",Dollars per Gallon,$/gal
4,PET.EMM_EPMMR_PTE_Y05SF_DPG.W,"San Francisco, CA Midgrade Reformulated Retail...",Dollars per Gallon,$/gal
5,PET.EMM_EPM0R_PTE_R5XCA_DPG.W,West Coast (PADD 5) Except California All Grad...,Dollars per Gallon,$/gal
6,PET.EMM_EPM0R_PTE_Y05LA_DPG.W,"Los Angeles, CA All Grades Reformulated Retail...",Dollars per Gallon,$/gal
7,PET.EMM_EPM0R_PTE_Y05SF_DPG.W,"San Francisco, CA All Grades Reformulated Reta...",Dollars per Gallon,$/gal
8,PET.EMM_EPM0R_PTE_Y35NY_DPG.W,New York Harbor All Grades Reformulated Retail...,Dollars per Gallon,$/gal
9,PET.EMM_EPM0R_PTE_Y44HO_DPG.W,"Houston, TX All Grades Reformulated Retail Gas...",Dollars per Gallon,$/gal


In [63]:
date_range_all = df.groupby('name')['end'].agg(['min', 'max'])
date_range_all_nonas = date_range_all[(date_range_all['min'].notna()) & (date_range_all['max'].notna())]
date_range_all_nonas.head(10)

,min,max
name,,
"\r\nU.S. Net Imports from Curacao of Liquified Petroleum Gases, Annual",2022,2022
"\r\nU.S. Net Imports from Curacao of Liquified Petroleum Gases, Monthly",202212,202212
"Alabama (with State Offshore) Associated-Dissolved Natural Gas, Reserves in Nonproducing Reservoirs, Wet, Annual",2021,2021
"Alabama (with State Offshore) Crude Oil Reserves in Nonproducing Reservoirs, Annual",2021,2021
"Alabama (with State Offshore) Natural Gas Liquids Lease Condensate, Reserves in Nonproducing Reservoirs, Annual",2021,2021
...,...,...
"Wyoming Total Gasoline Wholesale/Resale Price by All Sellers, Monthly",202203,202203
"Wyoming Total Gasoline Wholesale/Resale Price by Refiners, Annual",2021,2021
"Wyoming Total Gasoline Wholesale/Resale Price by Refiners, Monthly",202203,202203


In [64]:
# Return a count of the number of records by units 
df_series['units'].value_counts()

units
Thousand Barrels                     68599
Thousand Barrels per Day             67817
Dollars per Gallon                   17835
Thousand Gallons per Day             11784
Thousand Gallons                      6001
Barrels per Stream Day                1960
Million Barrels                       1261
Percent                                623
Barrels per Calendar Day               343
Dollars per Barrel                     304
Number of Elements                     287
Billion Cubic Feet                     168
Million Gallons                        117
Million Cubic Feet per Day              49
Short Tons per Day                      49
Thousand Barrels per Calendar Day       44
Thouand Barrels per Day                 32
Degree                                  32
Million Cubic Feet                      16
Thousand Feet                           14
Feet per Well                           12
Million Pounds                           7
Million Kilowatthours                    7
Thous

In [65]:
# Filter df to only include name containing 'Louisiana' and units in Dollars per Gallon
df_louisiana = df[df['name'].str.contains('Louisiana Total') & df['units'].str.contains('Dollars per Gallon')]
df_louisiana.head()

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,start,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202203, 3.09]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202202, 2.598]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202201, 2.331]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202112, 2.191]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202111, 2.332]",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,1984,2021,2022-04-01T10:51:55-04:00,"[1988, 0.552]",NaN,NaN,NaN,NaN,NaN
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,1984,2021,2022-04-01T10:51:55-04:00,"[1987, 0.566]",NaN,NaN,NaN,NaN,NaN
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,1984,2021,2022-04-01T10:51:55-04:00,"[1986, 0.511]",NaN,NaN,NaN,NaN,NaN
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,1984,2021,2022-04-01T10:51:55-04:00,"[1985, 0.832]",NaN,NaN,NaN,NaN,NaN


In [66]:
# Drop rows where 'data' is NaN or not a list
df_louisiana = df_louisiana.dropna(subset=['data'])
df_louisiana = df_louisiana[df_louisiana['data'].apply(lambda x: isinstance(x, list) and len(x) == 2)]
df_louisiana.head()

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,start,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202203, 3.09]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202202, 2.598]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202201, 2.331]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202112, 2.191]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202111, 2.332]",NaN,NaN,NaN,NaN,NaN


In [67]:
# Split 'data' column into 'date' and 'value'
df_louisiana[['date', 'value']] = pd.DataFrame(df_louisiana['data'].tolist(), index=df_louisiana.index)
df_louisiana.head()

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,...,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries,date,value
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202203, 3.09]",NaN,NaN,NaN,NaN,NaN,202203,3.090
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202202, 2.598]",NaN,NaN,NaN,NaN,NaN,202202,2.598
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202201, 2.331]",NaN,NaN,NaN,NaN,NaN,202201,2.331
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202112, 2.191]",NaN,NaN,NaN,NaN,NaN,202112,2.191
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202111, 2.332]",NaN,NaN,NaN,NaN,NaN,202111,2.332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,2021,2022-04-01T10:51:55-04:00,"[1988, 0.552]",NaN,NaN,NaN,NaN,NaN,1988,0.552
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,2021,2022-04-01T10:51:55-04:00,"[1987, 0.566]",NaN,NaN,NaN,NaN,NaN,1987,0.566
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,2021,2022-04-01T10:51:55-04:00,"[1986, 0.511]",NaN,NaN,NaN,NaN,NaN,1986,0.511
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,2021,2022-04-01T10:51:55-04:00,"[1985, 0.832]",NaN,NaN,NaN,NaN,NaN,1985,0.832


In [68]:
# Convert 'date' to datetime, coercing errors to NaT
df_louisiana['date'] = pd.to_datetime(df_louisiana['date'], format='%Y%m%d', errors='coerce')
# Convert 'value' to a numeric type, coercing errors to NaN
df_louisiana['value'] = pd.to_numeric(df_louisiana['value'], errors='coerce')
df_louisiana.head()

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,...,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries,date,value
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202203, 3.09]",NaN,NaN,NaN,NaN,NaN,NaT,3.090
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202202, 2.598]",NaN,NaN,NaN,NaN,NaN,NaT,2.598
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202201, 2.331]",NaN,NaN,NaN,NaN,NaN,NaT,2.331
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202112, 2.191]",NaN,NaN,NaN,NaN,NaN,2021-01-02,2.191
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202111, 2.332]",NaN,NaN,NaN,NaN,NaN,2021-01-01,2.332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,2021,2022-04-01T10:51:55-04:00,"[1988, 0.552]",NaN,NaN,NaN,NaN,NaN,NaT,0.552
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,2021,2022-04-01T10:51:55-04:00,"[1987, 0.566]",NaN,NaN,NaN,NaN,NaN,NaT,0.566
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,2021,2022-04-01T10:51:55-04:00,"[1986, 0.511]",NaN,NaN,NaN,NaN,NaN,NaT,0.511
116794,PET.EMA_EPM0_PWA_SLA_DPG.A,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,A,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,2021,2022-04-01T10:51:55-04:00,"[1985, 0.832]",NaN,NaN,NaN,NaN,NaN,NaT,0.832


In [69]:
df_louisiana.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4150 entries, 93062 to 116794
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   series_id           4150 non-null   object        
 1   name                4150 non-null   object        
 2   units               4150 non-null   object        
 3   f                   4150 non-null   object        
 4   unitsshort          4150 non-null   object        
 5   description         4150 non-null   object        
 6   copyright           4150 non-null   object        
 7   source              4150 non-null   object        
 8   iso3166             4150 non-null   object        
 9   geography           4150 non-null   object        
 10  start               4150 non-null   object        
 11  end                 4150 non-null   object        
 12  last_updated        4150 non-null   object        
 13  data                4150 non-null   object     

In [70]:
# Extract relevant columns and preprocess the data
df_louisiana['Date'] = pd.to_datetime(df_louisiana['date'])
df_louisiana['Price'] = df_louisiana['value']
df_louisiana = df_louisiana[['Date', 'Price', 'unitsshort', 'name', 'last_updated']].sort_values(by='Date').reset_index(drop=True)
df_louisiana.head()

,Date,Price,unitsshort,name,last_updated
0,1983-01-01,0.849,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,2022-06-01T10:50:11-04:00
1,1983-01-01,0.940,$/gal,Louisiana Total Gasoline Through Company Outle...,2022-06-01T10:50:11-04:00
2,1983-01-01,0.940,$/gal,Louisiana Total Gasoline Through Company Outle...,2022-06-01T10:50:11-04:00
3,1983-01-01,0.898,$/gal,Louisiana Total Gasoline Other End Users Price...,2022-06-01T10:50:11-04:00
4,1983-01-01,0.849,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,2022-06-01T10:50:11-04:00
...,...,...,...,...,...
4145,NaT,0.552,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,2022-04-01T10:51:55-04:00
4146,NaT,0.566,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,2022-04-01T10:51:55-04:00
4147,NaT,0.511,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,2022-04-01T10:51:55-04:00
4148,NaT,0.832,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,2022-04-01T10:51:55-04:00


In [71]:
date_range_la = df_louisiana.groupby('name')['Date'].agg(['min', 'max'])
date_range_la.head(10)

,min,max
name,,
"Louisiana Total Gasoline Bulk Sales Price by All Sellers, Annual",NaT,NaT
"Louisiana Total Gasoline Bulk Sales Price by All Sellers, Monthly",1994-01-01,2021-01-02
"Louisiana Total Gasoline DTW Sales Price by All Sellers, Annual",NaT,NaT
"Louisiana Total Gasoline DTW Sales Price by All Sellers, Monthly",1994-01-01,2021-01-02
"Louisiana Total Gasoline Other End Users Price by Refiners, Annual",NaT,NaT
"Louisiana Total Gasoline Other End Users Price by Refiners, Monthly",1983-01-01,2021-01-02
"Louisiana Total Gasoline Rack Sales Price by All Sellers, Annual",NaT,NaT
"Louisiana Total Gasoline Rack Sales Price by All Sellers, Monthly",1994-01-01,2021-01-02
"Louisiana Total Gasoline Retail Sales by All Sellers, Annual",NaT,NaT


In [72]:
# Load and prepare the dataset
def load_gas_price_data():
    df = pd.read_csv('data\processed\louisiana_tot_gasoline_wholesale_monthly.csv')
    df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Convert 'date' to datetime, coercing errors
    df = df[['date', 'value']].dropna()  # Keep only necessary columns and drop NA values
    return df

# Prepare data for long format and additional transformations
def prepare_data(df):
    df = df.sort_values('date')
    df['log_price'] = np.log(df['value'])
    df['price_change'] = df['log_price'].diff()
    return df

# Function to perform AutoARIMA forecasting
def forecast_prices(df, cutoff_date):
    # Filter the DataFrame based on the cutoff date
    df_filtered = df[df['date'] < pd.to_datetime(cutoff_date)]
    
    # Convert prices to log prices to stabilize variance
    df_filtered['log_price'] = np.log(df_filtered['value'])
    
    # Define the model
    model = SARIMAX(df_filtered['log_price'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    
    # Fit the model
    results = model.fit()
    
    # Generate future dates
    future_dates = pd.date_range(df_filtered['date'].max() + MonthEnd(1), periods=13, freq='M')
    
    # Forecast future log prices
    forecast_log_prices = results.forecast(steps=13)
    
    # Convert log prices back to regular prices
    forecast_prices = np.exp(forecast_log_prices)
    
    # Create a DataFrame for the forecasted prices
    forecast_df = pd.DataFrame({
        'date': future_dates,
        'forecast_price': forecast_prices
    })
    
    return forecast_df

# Visualization function
def plot_forecast(df, forecast_df):
    base = alt.Chart(df).encode(
        x='date:T',
        y='value:Q'
    ).properties(
        width=700,
        height=600
    )

    line = base.mark_line(color='blue', size=3)
    points = base.mark_point(color='red')

    forecast_chart = alt.Chart(forecast_df).mark_line(color='green').encode(
        x='date:T',
        y='forecast_price:Q'
    )

    return line + points + forecast_chart

In [73]:
df = load_gas_price_data()
df

,date,value
3,2021-01-02,2.191
4,2021-01-01,2.332
15,2020-01-02,1.307
16,2020-01-01,1.175
27,2019-01-02,1.636
...,...,...
3801,1985-01-01,0.846
3812,1984-01-02,0.787
3813,1984-01-01,0.816
3824,1983-01-02,0.819


In [74]:
df_prepared = prepare_data(df)
df_prepared.head()

,date,value,log_price,price_change
3825,1983-01-01,0.849,-0.163696,NaN
460,1983-01-01,0.849,-0.163696,0.000000
2551,1983-01-01,0.898,-0.107585,0.056111
931,1983-01-01,0.940,-0.061875,0.045710
1402,1983-01-01,0.940,-0.061875,0.000000
...,...,...,...,...
27,2019-01-02,1.636,0.492254,0.003674
16,2020-01-01,1.175,0.161268,-0.330986
15,2020-01-02,1.307,0.267734,0.106466
4,2021-01-01,2.332,0.846726,0.578992


In [75]:
forecast_df = forecast_prices(df, '2021-01-01')
forecast_df.tail()

C:\Users\LNshuti\AppData\Local\Temp\ipykernel_17420\892278828.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['log_price'] = np.log(df_filtered['value'])
c:\Users\LNshuti\AppData\Local\miniconda3\envs\oil-prices\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\LNshuti\AppData\Local\miniconda3\envs\oil-prices\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\LNshuti\AppData\Local\Temp\ipykernel_17420\892278828.py:30: FutureWarning: 'M' is deprecated and wil

,date,forecast_price
434,2020-01-31,0.848513
435,2020-02-29,0.847660
436,2020-03-31,0.806922
437,2020-04-30,0.837109
438,2020-05-31,0.835229
439,2020-06-30,0.844162
440,2020-07-31,0.772565
441,2020-08-31,0.786481
442,2020-09-30,0.739293
443,2020-10-31,0.767200


In [76]:
plot_forecast(df, forecast_df)

alt.LayerChart(...)

In [77]:
# # Save the model
# import joblib
# joblib.dump(model, 'models/oil_price_forecast_model.pkl')